# 1. Import thư viện

In [12]:
import cv2
import numpy as np
import os
from PIL import Image
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch.nn.parallel import DataParallel
from torch.cuda.amp import autocast, GradScaler
import torch.nn.functional as F
from torchvision.utils import save_image
import torchsummary
from tqdm import tqdm
from models.srcnn import *
from models.idn import *

from models.vdsr import *

from models.srresnet_ import *
from models.sr_model import *
from models.fsrcnn import *
from models.wdsr import *
from models.edsr import *

import time



In [13]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2. Tạo Mô hình SR

In [14]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# device = torch.device('cpu')

In [15]:
# edsr_vdsr = EDSR_vdsrfy()
edsr_orig = EDSR().to(device)
# torchsummary.summary(edsr_orig, (3,150, 150))
# edrn_canny = EDRN(use_canny=True).to(device)
# edrn_srresnet = EDRN(use_sobel=True).to(device)
srresnet = SRResNet().to(device)
# srcnn = SRCNN().to(device)
# vdsr = VDSR().to(device)
# fsrcnn = FSRCNN(scale_factor=4).to(device)


In [16]:

# edsr_orig.load_state_dict(torch.load('weight/best_edsr.pth', map_location=device))
# edsr_orig.load_state_dict(torch.load('weight/best_edsrx4_orig_model.pth', map_location=device))
# edrn_sobel.load_state_dict(torch.load('weight/best_sobel_srx4_model.pth', map_location=device))
# edrn_canny.load_state_dict(torch.load('weight/best_canny_srx4_model.pth', map_location=device))
srresnet.load_state_dict(torch.load('best_srresnet.pth', map_location=device))
# vdsr.load_state_dict(torch.load('weight/best_vdsr.pth', map_location=device))

/tmp/ipykernel_200791/82772818.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  srresnet.load_state_dict(torch.load('best_srresnet.pth', map_location=device))


<All keys matched successfully>

In [17]:
class ImageDataset(Dataset):
    def __init__(self, lr_dir, hr_dir):
        self.lr_files = sorted(os.listdir(lr_dir))
        self.hr_files = sorted(os.listdir(hr_dir))
        self.lr_dir = lr_dir
        self.hr_dir = hr_dir

    def __len__(self):
        return len(self.lr_files)

    def __getitem__(self, idx):
        lr_image = Image.open(os.path.join(self.lr_dir, self.lr_files[idx])).convert('RGB')
        hr_image = Image.open(os.path.join(self.hr_dir, self.hr_files[idx])).convert('RGB')
        
        # size = 32
        # transform_hr = transforms.Compose([
        #     transforms.ToPILImage(),
        #     transforms.CenterCrop((size*4, size*4)),
        #     transforms.ToTensor()
        # ])

        # transform_lr = transforms.Compose([
        #     transforms.ToPILImage(),
        #     transforms.CenterCrop((size, size)),
        #     # transforms.Resize((size*4, size*4)),
        #     transforms.ToTensor()
        # ])
        
        # lr_image = transform_lr(lr_image)
        # hr_image = transform_hr(hr_image)
        transform = transforms.Compose([
            # transforms.ToPILImage(),
            transforms.ToTensor()
        ])
        
        lr_image = transform(lr_image)
        hr_image = transform(hr_image)
        return lr_image, hr_image

# 3. Tạo Hyperparameter

In [18]:
# Đường dẫn tới bộ dữ liệu
train_lr_dir = 'Train/LR'
train_hr_dir = 'Train/HR'
valid_lr_dir = 'Test/LR'
valid_hr_dir = 'Test/HR'
# test_hr_dir  = '/kaggle/input/srdataset/sr_data/test/HR'
# test_lr_dir  = '/kaggle/input/srdataset/sr_data/test/LR'

# print(torch.cuda.memory_allocated())
# print(torch.cuda.memory_reserved())

In [19]:
from torch.amp import autocast, GradScaler
scaler = GradScaler()

# Khởi tạo dataset và dataloader
train_dataset = ImageDataset(train_lr_dir, train_hr_dir)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

valid_dataset = ImageDataset(valid_lr_dir, valid_hr_dir)
valid_loader = DataLoader(valid_dataset)

print(len(train_loader))

# Khởi tạo loss function
criterion = nn.MSELoss()

# Khởi tạo optimizers, schedulers cho từng mô hình
optim_edsr = optim.Adam(edsr_orig.parameters(), lr=1e-4, betas=(0.9, 0.999))
scheduler_edsr = optim.lr_scheduler.StepLR(optim_edsr, step_size=10**5, gamma=0.5)

optim_srresnet = optim.Adam(srresnet.parameters(), lr=1e-4, betas=(0.9, 0.999))
scheduler_srresnet = optim.lr_scheduler.StepLR(optim_srresnet, step_size=10**5, gamma=0.5)

# optim_vdsr = optim.Adam(vdsr.parameters(), lr=1e-4, betas=(0.9, 0.999))
# scheduler_vdsr = optim.lr_scheduler.StepLR(optim_vdsr, step_size=10**5, gamma=0.5)

# optim_fsrcnn = optim.Adam(fsrcnn.parameters(), lr=1e-4, betas=(0.9, 0.999))
# scheduler_fsrcnn = optim.lr_scheduler.StepLR(optim_fsrcnn, step_size=10**5, gamma=0.5)

12500


In [20]:
def calculate_psnr(img1, img2):
    mse = torch.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 1.0
    psnr = 20 * torch.log10(max_pixel / torch.sqrt(mse))
    return psnr.item()

In [21]:
print(device)

cuda


# 4. Training

In [22]:


# Hàm tính PSNR
def calculate_psnr(img1, img2):
    mse = torch.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 1.0
    psnr = 20 * torch.log10(max_pixel / torch.sqrt(mse))
    return psnr.item()

num_epochs = 14

best_psnr_edsr = float('-inf')
best_psnr_srresnet = float('-inf')
best_psnr_vdsr = float('-inf')
best_psnr_fsrcnn = float('-inf')
torch.cuda.empty_cache()

losses_edsr = []
losses_srresnet = []
losses_vdsr = []
losses_fsrcnn = []

avg_psnr_edsr = []
avg_psnr_srresnet = []
avg_psnr_vdsr = []
avg_psnr_fsrcnn = []

val_avg_psnr_edsr = []
val_avg_psnr_srresnet = []
val_avg_psnr_vdsr = []
val_avg_psnr_fsrcnn = []

patience = 50
epochs_no_improve = 0
log_file = open('training_log_models.txt', 'a')

for epoch in range(num_epochs):
    edsr_orig.train()
    srresnet.train()
    # vdsr.train()
    # fsrcnn.train()

    epoch_loss_edsr, psnr_values_edsr = 0, 0
    epoch_loss_srresnet, psnr_values_srresnet = 0, 0
    epoch_loss_vdsr, psnr_values_vdsr = 0, 0
    epoch_loss_fsrcnn, psnr_values_fsrcnn = 0, 0
    start_time = time.time()
    torch.cuda.empty_cache()
    # Training loop for each model
    for i, (lr_images, hr_images) in enumerate(tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')):
        lr_images = lr_images.to(device)
        hr_images = hr_images.to(device)

        # # # Train EDSR model
        # optim_edsr.zero_grad()
        # with autocast(device_type='cuda'):
        #     outputs_edsr = edsr_orig(lr_images)
        #     loss_edsr = criterion(outputs_edsr, hr_images)
        # psnr_edsr = calculate_psnr(outputs_edsr, hr_images)

        # scaler.scale(loss_edsr).backward()
        # scaler.step(optim_edsr)
        # scaler.update()
        # scheduler_edsr.step()

        # epoch_loss_edsr += loss_edsr.item()
        
        # psnr_values_edsr += psnr_edsr
        # while i < 12500/2:
        # Train SRResNet model
        optim_srresnet.zero_grad()
        with autocast(device_type='cuda'):
            outputs_srresnet = srresnet(lr_images)
            loss_srresnet = criterion(outputs_srresnet, hr_images)
        psnr_srresnet = calculate_psnr(outputs_srresnet, hr_images)

        scaler.scale(loss_srresnet).backward()
        scaler.step(optim_srresnet)
        scaler.update()
        scheduler_srresnet.step()

        epoch_loss_srresnet += loss_srresnet.item()
        psnr_values_srresnet += psnr_srresnet

        # Train RDN model
        # optim_vdsr.zero_grad()
        # with autocast(device_type='cuda'):
        #     outputs_vdsr = vdsr(lr_images)
        #     loss_vdsr = criterion(outputs_vdsr, hr_images)
        # psnr_vdsr = calculate_psnr(outputs_vdsr, hr_images)

        # scaler.scale(loss_vdsr).backward()
        # scaler.step(optim_vdsr)
        # scaler.update()
        # scheduler_vdsr.step()

        # epoch_loss_vdsr += loss_vdsr.item()
        # psnr_values_vdsr += psnr_vdsr

        # # Train FSRCNN model
        # optim_fsrcnn.zero_grad()
        # with autocast(device_type='cuda'):
        #     outputs_fsrcnn = fsrcnn(lr_images)
        #     loss_fsrcnn = criterion(outputs_fsrcnn, hr_images)
        # psnr_fsrcnn = calculate_psnr(outputs_fsrcnn, hr_images)

        # scaler.scale(loss_fsrcnn).backward()
        # scaler.step(optim_fsrcnn)
        # scaler.update()
        # scheduler_fsrcnn.step()

        # epoch_loss_fsrcnn += loss_fsrcnn.item()
        # psnr_values_fsrcnn += psnr_fsrcnn

    # Average losses and PSNRs
    avg_epoch_loss_edsr = epoch_loss_edsr / len(train_loader)
    avg_psnr_edsr_epoch = psnr_values_edsr / len(train_loader)
    losses_edsr.append(avg_epoch_loss_edsr)
    avg_psnr_edsr.append(avg_psnr_edsr_epoch)

    avg_epoch_loss_srresnet = epoch_loss_srresnet / len(train_loader)
    avg_psnr_srresnet_epoch = psnr_values_srresnet / len(train_loader)
    losses_srresnet.append(avg_epoch_loss_srresnet)
    avg_psnr_srresnet.append(avg_psnr_srresnet_epoch)

    # avg_epoch_loss_vdsr = epoch_loss_vdsr / len(train_loader)
    # avg_psnr_vdsr_epoch = psnr_values_vdsr / len(train_loader)
    # losses_vdsr.append(avg_epoch_loss_vdsr)
    # avg_psnr_vdsr.append(avg_psnr_vdsr_epoch)

    # avg_epoch_loss_fsrcnn = epoch_loss_fsrcnn / len(train_loader)
    # avg_psnr_fsrcnn_epoch = psnr_values_fsrcnn / len(train_loader)
    # losses_fsrcnn.append(avg_epoch_loss_fsrcnn)
    # avg_psnr_fsrcnn.append(avg_psnr_fsrcnn_epoch)

    # Validation for all models
    edsr_orig.eval()
    srresnet.eval()
    # vdsr.eval()
    # fsrcnn.eval()

    val_psnr_edsr, val_psnr_srresnet = 0, 0
    val_psnr_vdsr, val_psnr_fsrcnn = 0, 0

    with torch.no_grad():
        for (lr_images, hr_images) in valid_loader:
            lr_images = lr_images.cuda()
            hr_images = hr_images.cuda()

            # # Validate EDSR
            outputs_edsr = edsr_orig(lr_images)
            psnr_edsr = calculate_psnr(outputs_edsr, hr_images)
            val_psnr_edsr += psnr_edsr

            # Validate SRResNet
            outputs_srresnet = srresnet(lr_images)
            psnr_srresnet = calculate_psnr(outputs_srresnet, hr_images)
            val_psnr_srresnet += psnr_srresnet

            # # Validate RDN
            # outputs_vdsr = vdsr(lr_images)
            # psnr_vdsr = calculate_psnr(outputs_vdsr, hr_images)
            # val_psnr_vdsr += psnr_vdsr

            # # Validate FSRCNN
            # outputs_fsrcnn = fsrcnn(lr_images)
            # psnr_fsrcnn = calculate_psnr(outputs_fsrcnn, hr_images)
            # val_psnr_fsrcnn += psnr_fsrcnn

    val_avg_psnr_edsr_epoch = val_psnr_edsr / len(valid_loader)
    val_avg_psnr_edsr.append(val_avg_psnr_edsr_epoch)

    val_avg_psnr_srresnet_epoch = val_psnr_srresnet / len(valid_loader)
    val_avg_psnr_srresnet.append(val_avg_psnr_srresnet_epoch)

    # val_avg_psnr_rdn_epoch = val_psnr_rdn / len(valid_loader)
    # val_avg_psnr_rdn.append(val_avg_psnr_rdn_epoch)

    # val_avg_psnr_fsrcnn_epoch = val_psnr_fsrcnn / len(valid_loader)
    # val_avg_psnr_fsrcnn.append(val_avg_psnr_fsrcnn_epoch)

    # Save best model
    if val_avg_psnr_edsr_epoch > best_psnr_edsr:
        best_psnr_edsr = val_avg_psnr_edsr_epoch
        # torch.save(edsr_orig.state_dict(), 'best_edsr.pth')
        print(f"Saved EDSRR model with PSNR {best_psnr_edsr:.4f}")
    if val_avg_psnr_srresnet_epoch > best_psnr_srresnet:
        best_psnr_srresnet = val_avg_psnr_srresnet_epoch
        torch.save(srresnet.state_dict(), 'best_srresnet.pth')
        print(f"Saved SRResNet model with PSNR {best_psnr_srresnet:.4f}")

    # torch.save(edsr_orig.state_dict(), f'path/edsr_{epoch}.pth')
    torch.save(srresnet.state_dict(), f'path/srresnet_{epoch}.pth')
    # if val_avg_psnr_rdn_epoch > best_psnr_rdn:
    #     best_psnr_rdn = val_avg_psnr_rdn_epoch
    #     torch.save(rdn.state_dict(), 'best_rdn.pth')
    #     print(f"Saved RDN model with PSNR {best_psnr_rdn:.4f}")
    # if val_avg_psnr_fsrcnn_epoch > best_psnr_fsrcnn:
    #     best_psnr_fsrcnn = val_avg_psnr_fsrcnn_epoch
    #     torch.save(fsrcnn.state_dict(), 'best_fsrcnn.pth')

    print(f"Epoch [{epoch+1}/{num_epochs}] completed: EDSR Loss: {avg_epoch_loss_edsr:.4f}, PSNR: {avg_psnr_edsr_epoch:.4f}, Validation PSNR: {val_avg_psnr_edsr_epoch:.4f}, SRResNEt Loss: {avg_epoch_loss_srresnet:.4f}, PSNR: {avg_psnr_srresnet_epoch:.4f}, Validation PSNR: {val_avg_psnr_srresnet_epoch:.4f}")
# 
    log_file.write(f"Epoch {epoch+1}:  EDSR PSNR: {avg_psnr_edsr_epoch:.4f}, Validation PSNR: {val_avg_psnr_edsr_epoch:.4f}\n")
    log_file.write(f"              SRResNet PSNR: {avg_psnr_srresnet_epoch:.4f}, Validation PSNR: {val_avg_psnr_srresnet_epoch:.4f}\n")
    # log_file.write(f"Epoch {epoch+1}: RDN PSNR: {avg_psnr_rdn_epoch:.4f}, Validation PSNR: {val_avg_psnr_rdn_epoch:.4f}\n")
    # log_file.write(f"Epoch {epoch+1}: FSRCNN PSNR: {avg_psnr_fsrcnn_epoch:.4f}, Validation PSNR: {val_avg_psnr_fsrcnn_epoch:.4f}\n")

    log_file.flush()

log_file.close()


Epoch 1/14: 100%|██████████| 12500/12500 [17:16<00:00, 12.07batch/s]


Saved EDSRR model with PSNR 0.3977
Saved SRResNet model with PSNR 27.6994
Epoch [1/14] completed: EDSR Loss: 0.0000, PSNR: 0.0000, Validation PSNR: 0.3977, SRResNEt Loss: 0.0024, PSNR: 26.5301, Validation PSNR: 27.6994


Epoch 2/14: 100%|██████████| 12500/12500 [17:18<00:00, 12.04batch/s]


Epoch [2/14] completed: EDSR Loss: 0.0000, PSNR: 0.0000, Validation PSNR: 0.3977, SRResNEt Loss: 0.0024, PSNR: 26.5316, Validation PSNR: 27.6686


Epoch 3/14: 100%|██████████| 12500/12500 [17:21<00:00, 12.01batch/s]


...

Saved SRResNet model with PSNR 27.7892
Epoch [10/14] completed: EDSR Loss: 0.0000, PSNR: 0.0000, Validation PSNR: 0.3977, SRResNEt Loss: 0.0023, PSNR: 26.6477, Validation PSNR: 27.7892


Epoch 11/14: 100%|██████████| 12500/12500 [17:25<00:00, 11.96batch/s]


Epoch [11/14] completed: EDSR Loss: 0.0000, PSNR: 0.0000, Validation PSNR: 0.3977, SRResNEt Loss: 0.0023, PSNR: 26.6563, Validation PSNR: 27.7837


Epoch 12/14:  78%|███████▊  | 9804/12500 [13:39<03:45, 11.96batch/s]


KeyboardInterrupt: 

# 5. Testing